| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb) | [Lattice Strain](./09_lattice_strain.ipynb) | **Extensions:** | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |

## Getting Started with Geopandas


In this notebook we'll look at working with geochemistry in a spatial context, mainly looking at [`geopandas`](https://geopandas.org/en/stable/). We'll also look at how to bring some *simple* interactivity to your `matplotlib` figures, which could also be applied to any non-spatial case.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt

import pyrolite

In [ ]:
df = gpd.read_file("../data/regolith/CapricornSoilGeochem.shp") # stores it's own coordinate system, should be in crs="epsg:28350"
df = df.loc[df.Easting > 0]  # drop things which don't have locations

In [ ]:
df.crs

In [ ]:
df.columns

In [ ]:
df.columns = [c.replace("_ppm", "") for c in df.columns]
df.columns

In [ ]:
df.geometry

In [ ]:
df.geometry[0].x  # the x coordinate from the first point

In [ ]:
df.plot()

In [ ]:
colour_by = "Al"

In [ ]:
df.plot(
    c=df[colour_by]
)  # plot the data from our dataset, coloured by the column selected

In [ ]:
ax = df.plot(c=df[colour_by])
plt.colorbar(ax.collections[0], label=colour_by)  # add a colourbar for the variable

In [ ]:
from sklearn.decomposition import PCA
n_components = 5
pca_scores = gpd.GeoDataFrame(PCA(n_components=n_components).fit_transform(df.pyrochem.elements), columns=['PCA{}'.format(ix) for ix in range(n_components)], geometry=df.geometry)
pca_scores

In [ ]:
%matplotlib inline 
fig, ax = plt.subplots(1, n_components, sharex=True, sharey=True, figsize=(15, 3))
ax = list(ax.flat)
for a, c in zip(ax, pca_scores.columns.tolist()):
    a.set_title(c)
    pca_scores.loc[:, [c, 'geometry']].plot(c=pca_scores[c].values, ax=a, cmap=plt.get_cmap('cividis').copy())
    #plt.colorbar(a.collections[0], shrink=0.5)

## Basemaps with Contextily

In [ ]:
ax = df.plot(c=df[colour_by])
plt.colorbar(ax.collections[0], label=colour_by)
cx.add_basemap(ax, crs=df.crs.to_string())  # add a basemap under our dataset

In [ ]:
ax = df.plot(c=df[colour_by])
plt.colorbar(ax.collections[0], label=colour_by)
cx.add_basemap(
    ax, crs=df.crs.to_string(), source=cx.providers.Esri.WorldImagery, zoom=10
)  # add a basemap under our dataset, with the ESRI satellite imagery

In [ ]:
%matplotlib widget
# use an interactive backend for matplotlib
fig, ax = plt.subplots(1, figsize=(10, 6))  # create a figure with a specific size

df.plot(c=df[colour_by], marker="D", ax=ax)  # plot the Al_ppm data from our dataset

fig.colorbar(
    ax.collections[0], label=colour_by, shrink=0.8
)  # add a colourbar for the variable

ax.set(
    xlabel="Easting",
    ylabel="Northing",
    aspect="equal",
    xlim=(125000, 800000),
    ylim=(7.1e6, 7.65e6),
)
# modify some of the axis defaults, expand so we have broader context

cx.add_basemap(
    ax,
    crs=df.crs.to_string(),
    source=cx.providers.Esri.WorldImagery,
    zoom=10,
    attribution=False,
)  # add a basemap under our dataset

Add some more basic interactivity:

In [ ]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.colors

column = widgets.Dropdown(
    options=df.pyrochem.list_elements,
    value=colour_by,
    description="Column:",
    disabled=False,
)
cmap = widgets.Dropdown(
    options=plt.colormaps(),
    value="cividis",
    description="Colourmap:",
    disabled=False,
)

# remove zeros and below
df.pyrochem.elements = np.where(df.pyrochem.elements > 0, df.pyrochem.elements, np.nan) 

fig, ax = plt.subplots(1, figsize=(10, 6))

df.plot(c=df[colour_by], marker="D", ax=ax, norm=matplotlib.colors.LogNorm())
cb = fig.colorbar(ax.collections[0], label=colour_by, shrink=0.8)
ax.set(
    xlabel="Easting",
    ylabel="Northing",
    aspect="equal",
    xlim=(125000, 800000),
    ylim=(7.1e6, 7.65e6),
)
cx.add_basemap(
    ax,
    crs=df.crs.to_string(),
    source=cx.providers.Esri.WorldImagery,
    zoom=10,
    attribution=False,
)


def plot_column(column="Al", cmap="cividis"):
    ax.collections[0].cmap = plt.get_cmap(cmap).copy()
    ax.collections[0].set_array(df[column])
    ax.collections[0].set_clim(vmin=np.nanmin(df[column]), vmax=np.nanmax(df[column]))
    cb.update_normal(ax.collections[0])
    cb.set_label(column)
    fig.canvas.draw()


widget = widgets.interactive(plot_column, column=column, cmap=cmap)
display(
    widgets.VBox(
        [
            widgets.HBox(
                widget.children[:-1], layout=widgets.Layout(flex_flow="row wrap")
            ),
            widget.children[-1],
        ]
    )
)

--- 

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb) | [Lattice Strain](./09_lattice_strain.ipynb) | **Extensions:** | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |